# Exercise 9.1: The Wave Equation in Cylindrical Geometry

The wave equation in cylindrical geometry often leads to the eigenvalue problem:

$\left( \frac{ \mathrm{d}^2 } { \mathrm{d} r^2 } + \frac{1}{r} \frac{ \mathrm{d}}{\mathrm{d} r} \right) \Phi(r) = -k^2 \Phi(r)$,

with initial conditions $\Phi(r=0) = 1$ and $\Phi(r=1) = 0$. 

The analytical eigenfunctions are the regular cylindrical Bessel functions of order zero, and the eigenvalues correspond to the zeros of this function: 

$k_1 = 2.404826$, $k_2 = 5.520078$, $k_3 = 8.653728$, $k_4 = 11.791534$, ...

(a) Show that the substitution $\Phi = r^{-1/2} \phi$ changes this equation into one for which the Numerov algorithm is appropriate. 

(b) Modify the code given in Example 9.7 to solve this problem. Compare the numerical eigenvalues with the exact values. Note that you will not be able to start the integration at $r=0$. Try starting at e.g. $r=10^{-13}$, with integration step size $h=10^{-5}$. 

Note that you should expect $\mathcal{O}(\%)$ precision, but not better.

Solution:

Let's use sympy to do the substitution! 

In [26]:
from sympy import *

# define the independent variable and the constant k2:
r, k2, x= symbols('r k^2 x')


# define the functions:
Ph, ph, f, g= symbols('\Phi \phi f, g', cls=Function)

g = g(x)
f = exp(-x)*sin(x)**3
pprint(integrate(f, x))

#pprint(latex(diff(f,x,1)))

# tell sympy what the functions are: 
ph = ph(r)
Ph = r**(-1/2)* ph

# "substitute" into the differential equation: 
Eqn = diff(Ph,r, 2) + (1/r) * diff(Ph,r, 1) + k2 * Ph
pprint(latex(simplify(Eqn)))

     -x    3         -x    2                -x           2         -x    3   
  2⋅ℯ  ⋅sin (x)   3⋅ℯ  ⋅sin (x)⋅cos(x)   3⋅ℯ  ⋅sin(x)⋅cos (x)   3⋅ℯ  ⋅cos (x)
- ───────────── - ──────────────────── - ──────────────────── - ─────────────
        5                  5                      10                  10     
\frac{1.0 k^{2} \phi{\left(r \right)}}{r^{0.5}} + \frac{0.25 \phi{\left(r \right)}}{r^{2.5}} + \frac{1.0 \frac{d^{2}}{d r^{2}} \ph
i{\left(r \right)}}{r^{0.5}}


$x^{2} \frac{d^{2}}{d x^{2}} g{\left(x \right)} + 2 x \frac{d}{d x} g{\left(x \right)}$

So the equation reads: 

$\frac{1.0 k^{2} \phi{\left(r \right)}}{r^{0.5}} + \frac{0.25 \phi{\left(r \right)}}{r^{2.5}} + \frac{1.0 \frac{d^{2}}{d r^{2}} \phi{\left(r \right)}}{r^{0.5}} = 0$

or simplified: 

$\frac{ \mathrm{d}^2 \phi } { \mathrm{d} r^2 } = - (k^2 + \frac{1}{4} r^{-2}) \phi$.

We can now modify the code of Example 9.7 to get the eigenvalues. We need to replace $k^2$ by the appropriate function of $r$. 

In [8]:
import numpy as np
import math

# Numerov's algorithm (forward)
# takes as input the initial conditions y(0) and y'(0) as y0 and yp0
# h is the step size, the k-squared term (k2), the S term -- these are FUNCTIONS!
# the initial value of the independent variable x0, and the final value xf
# returns t,y as the solution arrays
def Numerov(k2, S, y0, yp0, h, x0, xf):
    """Returns the solution to a 2nd-order ODEs of the type: y'' + k^2 y = S(x) via the Numerov algorithm"""
    # the number of steps:
    N = int( (xf-x0)/h ) # needs to be an integer
    # get y1 via Taylor series:
    y1 = y0 + h * yp0
    # define the numpy arrays to return
    ya = np.zeros(N+1)
    xa = np.zeros(N+1)
    # set the first two values of the arrays:
    ya[0] = y0
    ya[1] = y1
    xa[0] = x0
    xa[1] = x0 + h
    # integrate via the Numerov algo:
    for n in range(1,N):
        x = x0 + n*h
        xa[n]=x
        h2dt = h**2/12 # appears often so let's just calculate it once!
        ya[n+1] = (2 * (1 - 5*h2dt * k2(x)) * ya[n] - (1 + h2dt *k2(x-h)) * ya[n-1] + h2dt*(S(x+h) + 10 * S(x) + S(x-h)))/((1 + h2dt * k2(x+h) ))    
    xa[N] = xf # set the last x value which is not set in the loop
    return xa,ya

In [9]:
# The bisection algorithm: 
# func should be a function for which we are trying to find the solution, in the form f(x)=0
# xmin and xmax should enclose the root (the function must change signs from xmin to xmax)
# Nmax is the number of evaluations
# prec is the required precision
def bisection(func, xmin, xmax, Nmax, prec): 
    """Function that implements the bisection algorithm for root finding"""
    n = 0 # number of steps taken
    val = 1E99 # the value of the equation, initialize to a large number
    root = math.nan # initialize the root to "not a number"
    while abs(val) > prec and n < Nmax: # loop terminates either when the max number of evals is reached or the precision is reached
        # get the equation values at the edges [xmin, xmax], 
        # and at the bisection point: 
        val = func((xmin+xmax)/2)
        valmax = func(xmax)
        valmin = func(xmin)
        # figure out in which of the two intervals there's a sign change:
        if val * valmax < 0: # sign change between bisection-xmax, set minimum to bisection
            xmin = (xmin+xmax)/2
        elif val * valmin < 0: # sign change between xmin-bisection, set max to bisection
            xmax = (xmin+xmax)/2
        n = n + 1
    if n > Nmax-1:
        print("Warning: maximum number of evaluations exceeded:", Nmax)
    root = (xmin+xmax)/2
    return root, n

The next function needs to be modified to take into account the new form of the equation.

In [10]:
# This function takes as input kest and yields phi(1) using the Numerov method
# It will form the input to our bisection search
def phi1(k):
    """Function that takes as input kest and yields phi(1) using the Numerov method"""
    # Numerov(k2, S, y0, yp0, h, x0, xf) <- function for reference 
    # you can define separate functions for k2 and S, but we can use the lambda method for brevity:
    x,y = Numerov(lambda x: k**2 + 0.25 * 1/(x**2), lambda x: 0, 0, 1, 1E-5, 1E-13, 1)
    return y[-1] # we only need the last element -> phi(1)

Check: 

In [14]:
print(phi1(2.404826)) # this should be non-zero!

0.0009459925003082181


In [19]:
Nmax = 1000
prec = 1E-5
keigen1, niter = bisection(phi1, 1, 3.0, Nmax, prec)
print(keigen1, 'fractional precision:', np.abs(2.404826-keigen1)/2.404826)
keigen2, niter = bisection(phi1, 4, 6.0, Nmax, prec)
print(keigen2, 'fractional precision:', np.abs(5.520078-keigen2)/5.520078)
keigen3, niter = bisection(phi1, 8, 9.0, Nmax, prec)
print(keigen3, 'fractional precision:', np.abs(8.653728-keigen3)/28.653728)
keigen4, niter = bisection(phi1, 10, 12.0, Nmax, prec)
print(keigen4, 'fractional precision:', np.abs(11.791534-keigen4)/11.791534)

2.521484375 fractional precision: 0.048510110502797334
5.642578125 fractional precision: 0.022191738051527566
8.796875 fractional precision: 0.004995754828132688
11.9296875 fractional precision: 0.01171632969891785
